In [ ]:
import cv2
import numpy as np

In [ ]:
def detect_ball(frame, ball_cascade, min_area=30):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    balls = ball_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), maxSize=(35, 35))
    
    for (x, y, w, h) in balls:
        area = w * h
        if area > min_area:
            ball_center_x = x + w // 2
            ball_center_y = y + h // 2
            radius = max(w, h) // 2
            cv2.circle(frame, (ball_center_x, ball_center_y), radius, (0, 255, 0), 2)
            return ball_center_x, ball_center_y
    
    return None

In [ ]:
def detect_players(frame, net_x):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 170, 180, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    player_counts = {
        'Brazil': 0,
        'Russia': 0
    }

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 400:
            x, y, w, h = cv2.boundingRect(contour)
            player_center_x = x + w // 2
            if net_x is not None and player_center_x < net_x:
                player_counts['Brazil'] += 1
            elif net_x is not None:
                player_counts['Russia'] += 1

    return frame, player_counts

In [ ]:
ball_cascade = cv2.CascadeClassifier("ball.xml")  

cap = cv2.VideoCapture("volleyball_match.mp4")

if not cap.isOpened():
    print("Error: Couldn't open video file.")
    exit()

net_x = None 

alpha = 0.1 

brazil_player_count_ema = 0
russia_player_count_ema = 0

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        ball_center = detect_ball(frame, ball_cascade)
        
        if ball_center:
            net_x = ball_center[0] 
        
        frame, player_counts = detect_players(frame, net_x)

        brazil_player_count_ema = (1 - alpha) * brazil_player_count_ema + alpha * player_counts['Brazil']
        russia_player_count_ema = (1 - alpha) * russia_player_count_ema + alpha * player_counts['Russia']
        
        cv2.putText(frame, f'Brazil: {int(brazil_player_count_ema)} players', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(frame, f'Russia: {int(russia_player_count_ema)} players', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        
        cv2.imshow('Ball and Player Detection', frame)
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()